<a href="https://colab.research.google.com/github/matsunagalab/ColabBTR/blob/main/advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 動的計画法の応用例

前回と同じように「ドライブにコピーを保存」してから始めてください。

今回は、動的計画法の応用例を2つ示します。これまで使ってきたソルバは使わずに、python文法だけで動的計画法を実装していきます。

## Levenshtein距離

In [1]:
def levenshtein_distance(s1, s2):
    m, n = len(s1), len(s2)

    # DPテーブルを初期化
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # 初期条件: 空文字列から変換するコスト
    for i in range(m + 1):
        dp[i][0] = i  # 削除コスト
    for j in range(n + 1):
        dp[0][j] = j  # 挿入コスト

    # DPテーブルを埋める
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:  # 一致の場合
                cost = 0
            else:  # 置換の場合
                cost = 1
            dp[i][j] = min(
                dp[i - 1][j] + 1,    # 削除
                dp[i][j - 1] + 1,    # 挿入
                dp[i - 1][j - 1] + cost  # 置換
            )

    # 最終的なLevenshtein距離
    return dp[m][n]

In [2]:
s1 = "kitten"
s2 = "sitting"
print("Levenshtein距離:", levenshtein_distance(s1, s2))  # 出力: 3

Levenshtein距離: 3


## アラインメント

In [5]:
def needleman_wunsch(seq1, seq2, match_score=1, mismatch_penalty=-1, gap_penalty=-1):
    m, n = len(seq1), len(seq2)

    # DPテーブルとトレースバックテーブルを初期化
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    traceback = [[None] * (n + 1) for _ in range(m + 1)]

    # 初期条件の設定（ギャップペナルティー）
    for i in range(1, m + 1):
        dp[i][0] = dp[i - 1][0] + gap_penalty
        traceback[i][0] = "up"
    for j in range(1, n + 1):
        dp[0][j] = dp[0][j - 1] + gap_penalty
        traceback[0][j] = "left"

    # DPテーブルの計算
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match = dp[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty)
            delete = dp[i - 1][j] + gap_penalty
            insert = dp[i][j - 1] + gap_penalty
            dp[i][j] = max(match, delete, insert)

            # トレースバックの記録
            if dp[i][j] == match:
                traceback[i][j] = "diag"
            elif dp[i][j] == delete:
                traceback[i][j] = "up"
            else:
                traceback[i][j] = "left"

    # アラインメントのトレースバック
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = m, n
    while i > 0 or j > 0:
        if traceback[i][j] == "diag":
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append(seq2[j - 1])
            i -= 1
            j -= 1
        elif traceback[i][j] == "up":
            aligned_seq1.append(seq1[i - 1])
            aligned_seq2.append("-")
            i -= 1
        elif traceback[i][j] == "left":
            aligned_seq1.append("-")
            aligned_seq2.append(seq2[j - 1])
            j -= 1

    # アラインメント結果を逆順にする
    aligned_seq1 = "".join(reversed(aligned_seq1))
    aligned_seq2 = "".join(reversed(aligned_seq2))

    return dp[m][n], aligned_seq1, aligned_seq2

In [6]:
seq1 = "GCTAGA"
seq2 = "AGCTAG"
score, aligned_seq1, aligned_seq2 = needleman_wunsch(seq1, seq2)

print("アラインメントスコア:", score)
print("アラインメント結果:")
print(aligned_seq1)
print(aligned_seq2)

アラインメントスコア: 3
アラインメント結果:
-GCTAGA
AGCTAG-
